In [22]:
import json
import os
import time
import requests
from typing import Optional, Dict
import base64
import mimetypes
import glob

class VeoVideoGenerator:
    def __init__(self, base_url: str = "https://api.thucchien.ai/gemini/v1beta", 
               api_key: str = "sk-Qyh3tDnMJmZUQZNJs4-P_Q"):
        self.base_url = base_url
        self.api_key = api_key
        self.headers = {
            'x-goog-api-key': api_key,
            'Content-Type': 'application/json'
        }
    
    def _send_generation_request(self, model: str, payload: Dict) -> Optional[str]:
        """Generic method to send a video generation request."""
        url = f"{self.base_url}/models/{model}:predictLongRunning"
        
        try:
            response = requests.post(url, headers=self.headers, json=payload)
            response.raise_for_status()

            data = response.json()
            operation_name = data.get('name')

            if operation_name:
                print(f"✅ Video generation started successfully! Operation: {operation_name}")
                return operation_name
            else:
                print("❌ No operation name returned.")
                print(f"Full Response: {json.dumps(data, indent=2)}")
                return None
        except requests.RequestException as e:
            print(f"❌ Failed to start video generation: {e}")
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    print(f"Error Details: {json.dumps(error_data, indent=2)}")
                except json.JSONDecodeError:
                    print(f"Error Response: {e.response.text}")
            return None

    def generate_video_from_text(self, prompt: str):
        """Starts video generation from a text prompt only."""
        print(f"🎬 Starting TEXT-TO-VIDEO generation for prompt: '{prompt}'")
        payload = {"instances": [{"prompt": prompt}]}
        # Assuming a different model for text-only, but using the same if not specified
        return self._send_generation_request("veo-3.0-generate-preview", payload)

    def generate_video_from_image(self, prompt: str, image_path: str, parameters: Dict):
        """Starts video generation from a single image, a prompt, and parameters."""
        print(f"🎬 Starting IMAGE-TO-VIDEO generation...")
        print(f"   Prompt: '{prompt}'")
        print(f"   Image: '{image_path}'")

        mime_type, _ = mimetypes.guess_type(image_path)
        if not mime_type:
            print(f"Could not determine MIME type for {image_path}. Aborting.")
            return None

        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

        payload = {
            "instances": [{
                "prompt": prompt,
                "image": {
                    "bytesBase64Encoded": encoded_string,
                    "mimeType": mime_type
                }
            }],
            "parameters": parameters
        }
        
        # Using the correct model name from your curl example
        return self._send_generation_request("veo-3.0-generate-001", payload)

    def wait_for_completion(self, operation_name: str, max_wait_time: int=600):
        # ... (This method remains the same as before) ...
        print(f"⏳ Waiting for video generation to complete...")
        operation_url = f"{self.base_url}/{operation_name}"
        start_time = time.time()
        poll_interval = 10 
        while time.time() - start_time < max_wait_time:
            try:
                print(f"   🔍 Polling status... ({int(time.time() - start_time)}s elapsed)")
                response = requests.get(operation_url, headers=self.headers)
                response.raise_for_status()
                data = response.json()
                if 'error' in data:
                    print("❌ Error during video generation process:")
                    print(json.dumps(data['error'], indent=2))
                    return None
                if data.get('done', False):
                    print("🎉 Video generation complete!")
                    try:
                        video_uri = data['response']['generateVideoResponse']['generatedSamples'][0]['video']['uri']
                        print(f"   Video URI: {video_uri}")
                        return video_uri
                    except KeyError:
                        print("❌ Could not extract video URI from final response.")
                        print(f"Full Response: {json.dumps(data, indent=2)}")
                        return None
                time.sleep(poll_interval)
            except requests.RequestException as e:
              print(f"❌ Error polling operation status: {e}")
              time.sleep(poll_interval)
        print(f"⏰ Timeout after {max_wait_time} seconds.")
        return None
    
    def download_video(self, video_uri: str, output_filename: str):
        # ... (This method remains the same as before) ...
        print(f"⬇️  Downloading video...")
        if video_uri.startswith("https://generativelanguage.googleapis.com/"):
            relative_path = video_uri.replace("https://generativelanguage.googleapis.com/", "")
        else:
            relative_path = video_uri
        if self.base_url.endswith("/v1beta"):
            base_path = self.base_url.replace("/v1beta", "/download")
        else:
            base_path = self.base_url
        litellm_download_url = f"{base_path}/{relative_path}"
        print(f"   Download URL: {litellm_download_url}")
        try:
            response = requests.get(litellm_download_url, headers=self.headers, stream=True, allow_redirects=True)
            response.raise_for_status()
            with open(output_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk: f.write(chunk)
            if os.path.exists(output_filename) and os.path.getsize(output_filename) > 0:
                print(f"✅ Video downloaded successfully!")
                print(f"   📁 Saved as: {output_filename}")
                print(f"   📏 File size: {os.path.getsize(output_filename) / (1024*1024):.2f} MB")
                return True
            else:
                print("❌ Downloaded file is empty or was not created.")
                return False
        except requests.RequestException as e:
            print(f"❌ Download failed: {e}")
            return False

    def run_image_to_video_workflow(self, prompt: str, image_path: str, parameters: Dict, output_filename: str = None):
        """Runs the complete image-to-video workflow."""
        if output_filename is None:
          timestamp = int(time.time())
          safe_prompt = "".join(c for c in prompt[:20] if c.isalnum()).rstrip()
          output_filename = f"veo_video_{safe_prompt}_{timestamp}.mp4"

        print("\n" + "=" * 60)
        print("🖼️  VEO IMAGE-TO-VIDEO WORKFLOW")
        print("=" * 60)
        
        operation_name = self.generate_video_from_image(prompt, image_path, parameters)
        if not operation_name:
            return False
        
        video_uri = self.wait_for_completion(operation_name)
        if not video_uri:
            return False
        
        success = self.download_video(video_uri, output_filename)

        if success:
          print("\n" + "=" * 60)
          print("🎉 WORKFLOW COMPLETE! 🎉")
          print(f"   Video saved as: {output_filename}")
          print("=" * 60)
        else:
            print("\n" + "=" * 60)
            print("❌ WORKFLOW FAILED")
            print("=" * 60)
        return success

# --- MAIN EXECUTION ---
base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/gemini/v1beta")
api_key = os.getenv("LITELLM_API_KEY", "sk-gjXpyfeUSridLeJhwRB2YA") # <-- IMPORTANT: REPLACE WITH YOUR KEY
print("🚀 Starting Veo Video Generation Example")
print(f"📡 Using LiteLLM proxy at: {base_url}")

generator = VeoVideoGenerator(base_url=base_url, api_key=api_key)

# --- Image-and-Text-to-Video Example ---
image_folder = "imgs"

# Hướng dẫn:
# 1. Tạo thư mục 'imgs' cùng cấp với file này.
# 2. Thêm một hoặc nhiều ảnh (.jpg, .png) vào đó. Code sẽ chọn ảnh ĐẦU TIÊN.
# 3. Chỉnh sửa `prompt_for_image` và `video_parameters` bên dưới.

if not os.path.exists(image_folder):
    os.makedirs(image_folder)
    print(f"\n✅ Created folder '{image_folder}'. Please add an image and run again.")
else:
    # Find the first supported image in the folder
    supported_extensions = ('*.png', '*.jpg', '*.jpeg')
    image_files = []
    for ext in supported_extensions:
        image_files.extend(glob.glob(os.path.join(image_folder, ext)))
    
    if image_files:
        first_image_path = sorted(image_files)[0]

        # ----> BẠN CÓ THỂ THAY ĐỔI CÁC THAM SỐ TẠI ĐÂY <----
        prompt_for_image = """
- Tham khảo ảnh mẫu, bạn hãy:
- Tạo đoạn video cảnh sau: dài 0-8s, CHÚ Ý LỖI TIẾNG VIỆT VÀ CHÍNH TẢ trong phông chữ, định dạng đầu ra mp4, kích thước 1920x1080 pixel.
- Phong cách truyền hình hiện đại, nền nhạc nhẹ, đồ họa sinh động, có chuyển cảnh mượt.
- Nội dung bản tin tổng hợp và phân tích điểm nổi bật về tình hình phát triển AI tại Việt Nam (tính đến tháng 10/2025).
- Có MC ảo giọng chuẩn, phong thái chuyên nghiệp, nói tiếng Việt chuẩn, tự nhiên xuất hiện trong trường quay hiện đại .
- Có dòng chữ “Đây là sản phẩm tham dự cuộc thi AI Thực Chiến” đặt ở góc phải trên cùng.

- Hình ảnh trung tâm: Biểu tượng chiếc khóa và dữ liệu
- Có tiêu đề bản tin: "Tổng hợp điểm nổi bật về tình hình phát triển AI tại Việt Nam (tính đến tháng 10/2025)". 
Lời dẫn của MC (bạn có thể nói thêm sao cho đủ thời gian và biến đổi sao cho chuyên nghiệp):
"Tiếp theo là chúng ta sẽ nói về thách thức và triển vọng đó là về bảo mật dữ liệu cá nhân và thiếu hụt chuyên gia AI trình độ cao”



"""
        
        video_parameters = {
            "negativePrompt": "blurry, low quality, static",
            "aspectRatio": "16:9",
            "resolution": "1080p",
        }
        # ---------------------------------------------------------

        generator.run_image_to_video_workflow(
            prompt=prompt_for_image,
            image_path=first_image_path,
            parameters=video_parameters
        )
    else:
        print(f"\n⚠️ Folder '{image_folder}' is empty. Please add an image to run this example.")

🚀 Starting Veo Video Generation Example
📡 Using LiteLLM proxy at: https://api.thucchien.ai/gemini/v1beta

🖼️  VEO IMAGE-TO-VIDEO WORKFLOW
🎬 Starting IMAGE-TO-VIDEO generation...
   Prompt: '
- Tham khảo ảnh mẫu, bạn hãy:
- Tạo đoạn video cảnh sau: dài 0-8s, CHÚ Ý LỖI TIẾNG VIỆT VÀ CHÍNH TẢ trong phông chữ, định dạng đầu ra mp4, kích thước 1920x1080 pixel.
- Phong cách truyền hình hiện đại, nền nhạc nhẹ, đồ họa sinh động, có chuyển cảnh mượt.
- Nội dung bản tin tổng hợp và phân tích điểm nổi bật về tình hình phát triển AI tại Việt Nam (tính đến tháng 10/2025).
- Có MC ảo giọng chuẩn, phong thái chuyên nghiệp, nói tiếng Việt chuẩn, tự nhiên xuất hiện trong trường quay hiện đại .
- Có dòng chữ “Đây là sản phẩm tham dự cuộc thi AI Thực Chiến” đặt ở góc phải trên cùng.

- Hình ảnh trung tâm: Biểu tượng chiếc khóa và dữ liệu
- Có tiêu đề bản tin: "Tổng hợp điểm nổi bật về tình hình phát triển AI tại Việt Nam (tính đến tháng 10/2025)". 
Lời dẫn của MC (bạn có thể nói thêm sao cho đủ thời gian